In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optm

## Create Model
* ### nn.Module
  最基本的create model方式,彈性最大,需要override __init__() , __forward__()。<br>
  缺點: 如果layer很多,寫forward很麻煩,通常是搭配Sequential和ModuleList寫好的block使用。
* ### nn.Sequential
  如果要建構的模型很單純,不需要定義特殊的forward,使用Sequential。(Sequential是Module的subclass,已經內建forward) <br>
  注意1:因為nn.Sequential已經內建forward,所以必須確定上下兩層layer之間的input和output是否吻合。<br>
  注意2:可以直接使用nn.Sequential來定義整個model,但是彈性差(無法自己定義forward),通常是用nn.Sequential來定義block並和nn.Module一起使用。<br>
  Ex. (conv2d + Relu) * 2的架構為例 
  * 方法一:直接將torch.nn中的元件當參數傳入 <br>
    缺點:layer name無法指定,會用預設的數字。
  * 方法二:用OrderedDict當參數傳入。<br>
    優點：可指定layer name <br>
  
    

In [2]:
#方法一
class net1(nn.Module):
    def __init__(self):
        super(net1, self).__init__()
        self.seq = nn.Sequential(nn.Conv2d(1, 20, 3, 1), nn.ReLU(), nn.Conv2d(20, 20, 3, 1), nn.ReLU())
    def forward(self, x):
        x = self.seq(x)
        return x
    #def forward(self, x):
    #    for s in self.seq:
    #        x = s(x)
    #    return x
net = net1()
print(net)

net1(
  (seq): Sequential(
    (0): Conv2d(1, 20, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(20, 20, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
  )
)


In [3]:
#方法二
from collections import OrderedDict
class net2(nn.Module):
    def __init__(self):
        super(net2, self).__init__()
        self.seq = nn.Sequential(OrderedDict([
            ('conv1', nn.Conv2d(1, 20, 3, 1)),
            ('relu1', nn.ReLU()),
            ('conv2', nn.Conv2d(20, 20 ,3,1)),
            ('relu2', nn.ReLU())
        ]))
    def forward(self, x):
        x = self.seq(x)
        return x
    #def forward(self, x):
    #    for s in self.seq:
    #        x = s(x)
    #    return x
net = net2()
print(net)

net2(
  (seq): Sequential(
    (conv1): Conv2d(1, 20, kernel_size=(3, 3), stride=(1, 1))
    (relu1): ReLU()
    (conv2): Conv2d(20, 20, kernel_size=(3, 3), stride=(1, 1))
    (relu2): ReLU()
  )
)


* ### nn.ModuleList
  nn.ModuleList是一個存放不同module的容器,並會把module的parameter(weight, bias)存到network中。<br>
  注意1:ModuleList沒有內建forward所以,存在ModuleList內的module的順序沒有任何太大意義。<br>
  注意2:操作方式和python的list類似,有extend和append,但是一般python的list並不會將module的parameter(weight, bias)存到network中。<br>
  使用時機1:如果某個block或model重複性的東西太多,可用modulelist來完成(透過for迴圈)，但可用nn.Sequential實現。<br>
  使用時機2:如果像是ResNet的short cut或是FCN中的skip architecture需要和前面層做concate或者add的,使用modulelist較佳。
  
* ### 補充:torch.nn和torch.nn.functional比較 
  [torch.nn和torch.nn.functional]( https://www.zhihu.com/question/66782101)

In [4]:
#使用時機1 範例(nn.ModuleList Version)
class net3(nn.Module):
    def __init__(self):
        super(net3, self).__init__()
        self.list = nn.ModuleList([nn.Linear(10, 10) for i in range(5)])
    def forward(self, x):
        for layer in self.list:
            x = layer(x)
        return x

net = net3()
print(net)  

#使用時機1 範例(nn.Sequential Version)
class net4(nn.Module):
    def __init__(self):
        super(net4, self).__init__()
        self.list = [nn.Linear(10, 10) for i in range(5)]
        
        # 重要!!! : *這個符號可以將list拆開成一個一個element並按造順序
        self.seq = nn.Sequential(*self.list)
        self.seq2 = nn.Sequential()
        for l in self.list:
            self.seq2.add_module('linear_'+ str(self.list.index(l)), l)
    def forward(self, x):
        x = seq(x)
        x = seq2(x)
        return x
    
net = net4()
print(net)  

#使用時機2 範例
def conv_block(nums_layer):
    block =[]
    for i in range(nums_layer):
        if (i+1) % 3 == 0:
            block.append(nn.Conv2d(20,30,3,1))
        else:
            block.append(nn.Conv2d(20,20,3,1))
    return block 
class net5(nn.Module):
    def __init__(self):
        super(net5, self).__init__()
        self.cnn_list = nn.ModuleList(conv_block(3))
        self.trace = []
    def forward(self, x):
        for layer in self.cnn_list:
            x = layer(x)
            self.trace.append(x)
        return x
net = net5()
print(net)
input_tensor  = torch.randn(100, 20, 10, 10) # input batch size: 100 , channel:20, h:10, w:10
output = net(input_tensor)
for each in net.trace:
    print(each.shape)

net3(
  (list): ModuleList(
    (0): Linear(in_features=10, out_features=10, bias=True)
    (1): Linear(in_features=10, out_features=10, bias=True)
    (2): Linear(in_features=10, out_features=10, bias=True)
    (3): Linear(in_features=10, out_features=10, bias=True)
    (4): Linear(in_features=10, out_features=10, bias=True)
  )
)
net4(
  (seq): Sequential(
    (0): Linear(in_features=10, out_features=10, bias=True)
    (1): Linear(in_features=10, out_features=10, bias=True)
    (2): Linear(in_features=10, out_features=10, bias=True)
    (3): Linear(in_features=10, out_features=10, bias=True)
    (4): Linear(in_features=10, out_features=10, bias=True)
  )
  (seq2): Sequential(
    (linear_0): Linear(in_features=10, out_features=10, bias=True)
    (linear_1): Linear(in_features=10, out_features=10, bias=True)
    (linear_2): Linear(in_features=10, out_features=10, bias=True)
    (linear_3): Linear(in_features=10, out_features=10, bias=True)
    (linear_4): Linear(in_features=10, out_fe

### VGG 16 & VGG 19

cfg: 定義Feature extraction layer架構 <br>

#### [def]<br>
vgg16:對外的api接口<br>
_vgg: create model 並且載入 pretrain weight<br>
make_layers: create model<br>

#### [class] <br>
vgg: 將Feature extraction layer和classifier串連起來，並定義整個forward



In [5]:
model_urls = {
    'vgg11': 'https://download.pytorch.org/models/vgg11-bbd30ac9.pth',
    'vgg13': 'https://download.pytorch.org/models/vgg13-c768596a.pth',
    'vgg16': 'https://download.pytorch.org/models/vgg16-397923af.pth',
    'vgg19': 'https://download.pytorch.org/models/vgg19-dcbb9e9d.pth',
    'vgg11_bn': 'https://download.pytorch.org/models/vgg11_bn-6002323d.pth',
    'vgg13_bn': 'https://download.pytorch.org/models/vgg13_bn-abd245e5.pth',
    'vgg16_bn': 'https://download.pytorch.org/models/vgg16_bn-6c64b313.pth',
    'vgg19_bn': 'https://download.pytorch.org/models/vgg19_bn-c79401a0.pth',
}
cfg = {
      'vgg16':[64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
      'vgg19':[64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M']
}
class vgg(nn.Module):
    def __init__(self, feature, num_classes=1000, init_weights=True):
        super(vgg, self).__init__()
        self.feature = feature
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.classifier = nn.Sequential(nn.Linear(512 * 7 * 7, 4096),
                                        nn.ReLU(inplace=True),
                                        nn.Dropout(0.25),
                                        nn.Linear(4096, 4096),
                                        nn.ReLU(inplace=True),
                                        nn.Dropout(0.25),
                                        nn.Linear(4096, num_classes)
                                       )
        
    def forward(self, x):
        x = self.feature(x)
        #將feature map的輸出整理成 7*7(因為input可能是不同size的image)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x
        

In [6]:
def make_layers(arch, batch_norm):
    layers = nn.Sequential()
    in_channel = 3
    conv_idx = 0
    bn_idx = 0
    pool_idx = 0
    relu_idx = 0
    #使用.add_module來串接model是為了對每一個layer命名，如果不想命名可以直接將每一個layer塞到一個list
    #並使用nn.Sequential(*list)快速串連模型
    for l in cfg[arch[:5]]:
        if l == 'M':
            layers.add_module('max_pooling_' + str(pool_idx), nn.MaxPool2d(kernel_size=2, stride=2))
            pool_idx += 1
        else:
            conv2d = nn.Conv2d(in_channel, l, 3, 1, padding=1)
            layers.add_module('conv2d_' + str(conv_idx), conv2d)
            conv_idx += 1
            if batch_norm:
                layers.add_module('batch_norm_' + str(bn_idx), nn.BatchNorm2d(l))
                layers.add_module('relu_' + str(relu_idx), nn.ReLU(inplace=True))
                bn_idx += 1
                relu_idx += 1
            else:
                layers.add_module('relu_' + str(relu_idx), nn.ReLU(inplace=True))
                relu_idx += 1
            in_channel = l
    return layers

from torch.hub import load_state_dict_from_url
def _vgg(arch, pretrain, batch_norm, **kwargs):
    if pretrain:
        kwargs['init_weights'] = False
    model = vgg(make_layers(arch, batch_norm), **kwargs)
    if pretrain:
        state_dict = load_state_dict_from_url(model_urls[arch])
        model.load_state_dict(state_dict)

def vgg16Bn(pretrain=False, **kwargs):
    return _vgg('vgg16_bn', pretrain, True)

In [7]:
vgg16Bn = vgg(make_layers('vgg16_bn', batch_norm=True), 14, False)
print(vgg16Bn)

vgg(
  (feature): Sequential(
    (conv2d_0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (batch_norm_0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu_0): ReLU(inplace=True)
    (conv2d_1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (batch_norm_1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu_1): ReLU(inplace=True)
    (max_pooling_0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2d_2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (batch_norm_2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu_2): ReLU(inplace=True)
    (conv2d_3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (batch_norm_3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu_3): ReLU(inplace=True)
    (max_p

In [8]:
from torchvision import transforms
train_trm = transforms.Compose([transforms.Resize((224, 224)),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.4631, 0.4959, 0.4289],std=[0.2755, 0.2713, 0.3078])
            ])
val_trm = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.4631, 0.4959, 0.4289],std=[0.2755, 0.2713, 0.3078])
])

In [10]:
# ImageFolder
import torchvision.datasets as dset
train_root_dir = '/Users/lionl771128/Documents/AI_Projdct/dataset1122/train/image/tree'
val_root_dir = '/Users/lionl771128/Documents/AI_Projdct/dataset1122/test/image/tree'
train_set_image_folder = dset.ImageFolder(root=train_root_dir, transform=train_trm)
val_set_image_folder = dset.ImageFolder(root=val_root_dir, target_transform=val_trm)
len(train_set_image_folder)

1815

In [12]:
from torch.utils.data import DataLoader
train_loader = DataLoader(dataset=train_set_image_folder,
                          batch_size=16,
                          shuffle=True,
                          num_workers=4
                         )

test_loader = DataLoader(dataset=val_set_image_folder,
                        batch_size=16,
                        shuffle=True,
                        num_workers=4)

In [13]:
'''
計算mean以及std，供transforms.Normalize使用
'''
import numpy as np
def get_mean_std(dataset, batch_size):
    train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    #train = iter(train_loader).next()[0]
    s = torch.Tensor()
    for data, target in train_loader:
        s = torch.cat((s, data), 0)
      
    mean = torch.mean(s, (0, 2, 3))
    std = torch.std(s, (0, 2, 3))
    print(mean, std)

# train_set_ = leaf_tree(root_dict='C:/Users/scott/AI_Project/dataset1122',
#                       sub_dict ='tree',
#                       transforms=transforms.Compose([transforms.Resize((446, 446)),
#                                 transforms.ToTensor()])
#                      )
# get_mean_std(train_set_, batch_size=64)

In [ ]:
# from apex import amp
# model, optimizer = amp.initialize(model, optimizer, opt_level="O1") # 这里是“欧一”，不是“零一”
# with amp.scale_loss(loss, optimizer) as scaled_loss:
#     scaled_loss.backward()

In [ ]:
# from tqdm import tqdm 
from tqdm.notebook import tqdm
from torch.optim.lr_scheduler import StepLR
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = vgg(make_layers(arch='vgg16_bn', batch_norm=True), 14, True).to(device)
optimizer = optm.Adam(model.parameters())
schedule = StepLR(optimizer, step_size=3, gamma=0.1)
nums_epoch = 100

# from apex import amp
# model, optimizer = amp.initialize(model, optimizer, opt_level="O1") # 这里是“欧一”，不是“零一”
for epoch in range(nums_epoch):
    idx = 0
    model.train()
    with tqdm(train_loader, leave=True, total=len(train_loader)) as t:
        t.set_description('Epoch %d/%d' %(epoch+1, nums_epoch))
        for data, target in t:
            optimizer.zero_grad()
            data, target = data.to(device), target.to(device)
            output = model(data)
            print(output.shape)
            print(output[0])
            print(output.argmax(dim=1, keepdim=True))
            p = output.argmax(dim=1, keepdim=True)
            print(p.eq(target).sum().item())
            print(p.eq(target))
            print(target)
            print(len(train_loader.dataset))
            print(len(train_loader))
            loss = F.cross_entropy(output, target)
#             with amp.scale_loss(loss, optimizer) as scaled_loss:
#                 scaled_loss.backward()
            loss.backward()
            optimizer.step()
            idx += 1
            if idx % len(train_loader) != 0:
                t.set_postfix(train_loss=loss.item())
            else:
                model.eval()
                correct = 0
                val_loss = 0
                for data, target in val_loader:
                    with torch.no_grad():
                        data, target = data.to(device), target.to(device)
                        output = model(data)
                        val_loss += F.cross_entropy(output, target)
                        pred = output.argmax(dim=1)
                        correct += pred.eq(target).sum().item()
                        
                accuracy = 100. * correct / len(val_loader.dataset)
                t.set_postfix(train_loss=loss.item(),val_loss=val_loss.item(), accuracy=accuracy)
                

create Dataset

In [ ]:
from torch.utils.data import Dataset
from glob import glob
import os, io
import pandas as pd
from PIL import Image
class leaf_tree(Dataset):
    def __init__(self, root_dict, sub_dict, transforms, train=True):
        self.transforms = transforms
        dir_type = 'train' if train else 'test'
        paths = []
        labels = []
        label = 0
        folders = glob(root_dict + '/' + dir_type + '/image/' + sub_dict + '/*')
        for f in folders:
            p = glob(f + '/*.[jJ][pP][gG]')
            paths.extend(p)
            labels.extend([label] * len(p))
            label += 1
        image_df = pd.DataFrame({
            'path':paths,
            'label':labels
        })
        self.image_df = image_df
    def __len__(self):
        return len(self.image_df)
    def __getitem__(self, idx):
        path = self.image_df.iloc[idx, 0]
        label = self.image_df.iloc[idx, 1]
        image = Image.open(path)
        if self.transforms is not None:
            image = self.transforms(image)
        return image, label

#'https://medium.com/@rowantseng/pytorch-%E8%87%AA%E5%AE%9A%E7%BE%A9%E8%B3%87%E6%96%99%E9%9B%86-custom-dataset-7f9958a8ff15'

In [ ]:
train_set = leaf_tree(root_dict='C:/Users/scott/AI_Project/dataset1122',
                      sub_dict ='tree',
                      transforms=train_trm
                     )
val_set = leaf_tree(root_dict='C:/Users/scott/AI_Project/dataset1122',
                    sub_dict='tree',
                    transforms=val_trm,
                    train=False
                   )

In [ ]:
from torch.utils.data import DataLoader
train_loader = DataLoader(dataset=train_set,
                          batch_size=16,
                          shuffle=True,
                          num_workers=0
                         )
val_loader = DataLoader(dataset=val_set,
                        batch_size=16,
                        shuffle=True,
                        num_workers=0)